This demo shows pretrain `TrxEncoder` with MaskedLanguageModel task.

## Setup

In [ ]:
%load_ext autoreload
%autoreload 2

# import logging
import torch
import pytorch_lightning as pl
# import warnings

# warnings.filterwarnings('ignore')
# logging.getLogger("pytorch_lightning").setLevel(logging.ERROR)

## Data preproccessing

In [ ]:
import os
import pandas as pd

data_path = 'data/'

source_data = pd.read_csv('https://huggingface.co/datasets/dllllb/age-group-prediction/resolve/main/transactions_train.csv.gz?download=true', compression='gzip')
source_data.head(2)

In [ ]:
from ptls.preprocessing import PandasDataPreprocessor

preprocessor = PandasDataPreprocessor(
    col_id='client_id',
    col_event_time='trans_date',
    event_time_transformation='none',
    cols_category=['small_group'],
    cols_numerical=['amount_rur'],
    return_records=True,
)

In [ ]:
%%time

dataset = preprocessor.fit_transform(source_data)

In [ ]:
import pickle

with open('preprocessor.p', 'wb') as f:
    pickle.dump(preprocessor, f)

In [ ]:
dataset = sorted(dataset, key=lambda x: x['client_id'])

In [ ]:
# Load targets
df_target = pd.read_csv('https://huggingface.co/datasets/dllllb/age-group-prediction/resolve/main/train_target.csv?download=true')
df_target.set_index('client_id', inplace=True)
df_target.rename(columns={"bins": "target"}, inplace=True)
df_target.head(5)

In [ ]:
# Add targets to tdataset
print(dataset[0].keys())

for el in dataset:
    el['target'] = df_target['target'][el['client_id']]

print(dataset[0].keys())

In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(dataset, test_size=0.2, random_state=42)
train, valid = train_test_split(train, test_size=4000, random_state=42)

len(train), len(valid), len(test)

In [ ]:
import pickle

with open('train_list', 'wb') as h:
      pickle.dump(train, h)
        
with open('valid_list', 'wb') as h:
      pickle.dump(valid, h)

with open('test_list', 'wb') as h:
      pickle.dump(test, h)

## Embedding training

Model training in our framework organised via pytorch-lightning (pl) framework.
The key parts of neural networks training in pl are: 

    * model (`pytorch_lightning.LightningModule`)
    * data loader (`torch.utils.data.DataLoader`)
    * trainer (`pytorch_lightning.Trainer`)
    
For futher details check https://pytorchlightning.ai/

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pickle

with open('train_list', 'rb') as h:
      train = pickle.load(h)
        
with open('valid_list', 'rb') as h:
      valid = pickle.load(h)

with open('test_list', 'rb') as h:
      test = pickle.load(h)

### Data loader

In [3]:
from ptls.data_load.datasets import MemoryMapDataset
from ptls.data_load.iterable_processing import FeatureFilter
from ptls.frames.bert import MlmDataset
from ptls.frames.tabformer.tabformer_dataset import TabformerDataset
from ptls.frames import PtlsDataModule

mlm_dm = PtlsDataModule(
    train_data=TabformerDataset(
        MemoryMapDataset(
            data=train,
        ),
        min_len=100, max_len=110
    ),
    valid_data=MlmDataset(
        MemoryMapDataset(
            data=valid,
            i_filters=[
                FeatureFilter(),
            ],
        ),
        min_len=200, max_len=256
    ),
    train_num_workers=16,
    train_batch_size=128,
)

### Model definition

In [4]:
import torch
from ptls.nn import TrxEncoder, LongformerEncoder, TabFormerFeatureEncoder, TransformerEncoder
from ptls.frames.bert import MLMPretrainModule
from ptls.frames.tabformer.tabformer_module import TabformerPretrainModule
from ptls.nn import PBLinear, PBL2Norm, PBLayerNorm

In [5]:
trx_encoder_params = dict(
    embeddings_noise=0,
#     numeric_values={'amount_rur': 'identity'},
    embeddings={
        'trans_date': {'in': 800, 'out': 16},
        'small_group': {'in': 250, 'out': 16},
    },
)

trx_encoder = TrxEncoder(**trx_encoder_params)

feature_encoder = TabFormerFeatureEncoder(2, 16)

seq_encoder = TransformerEncoder(input_size=32, n_heads=2, n_layers=1)

In [6]:
mlm_module = TabformerPretrainModule(
    trx_encoder=trx_encoder,
#     torch.nn.Sequential(
#         trx_encoder,
#         PBLinear(trx_encoder.output_size, 64),
#         PBL2Norm(),
#     ),
    seq_encoder=seq_encoder,
    feature_encoder=feature_encoder,
    hidden_size=64,
    
    total_steps=30000,

    mask_prob=0.15
)

/home/alge/.local/share/virtualenvs/ptls-experiments-0GRH_plu/src/pytorch-lifestream/ptls/frames/tabformer/tabformer_module.py:77: UserWarning: With Tabformer model set `in` value in `embeddings`parameter of `trx_encoder` equal to actual number of unique feature values + 1
  warnings.warn("With Tabformer model set `in` value in `embeddings`parameter of `trx_encoder` equal to actual number of unique feature values + 1")


### Trainer

In [9]:
import numpy as np
import torch
import pytorch_lightning as pl

import logging

trainer = pl.Trainer(
    max_steps=120000,  # 12000
    gpus=1 if torch.cuda.is_available() else 0,
    enable_progress_bar=True,
    callbacks=[
        pl.callbacks.LearningRateMonitor(logging_interval='step'),
        pl.callbacks.ModelCheckpoint(every_n_train_steps=2000, save_top_k=-1),
    ]
)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [8]:
mlm_module.hparams

"hidden_size":  64
"mask_prob":    0.15
"max_lr":       0.001
"norm_predict": False
"pct_start":    0.1
"total_steps":  30000
"weight_decay": 0.0

### Training 

In [ ]:
%%time
print(f'logger.version = {trainer.logger.version}')
trainer.fit(mlm_module, mlm_dm)
print(trainer.logged_metrics)

### Save sequence encoder for other experiments

In [ ]:
torch.save(mlm_module.trx_encoder.state_dict(), "mlm-emb.pt")

## Finetuning 

### Data module

In [ ]:
from ptls.data_load.datasets import MemoryMapDataset
from ptls.data_load.iterable_processing import SeqLenFilter, FeatureFilter
from ptls.frames.supervised import SeqToTargetDataset
from ptls.frames import PtlsDataModule

def get_dataset(data):
    return SeqToTargetDataset(
        MemoryMapDataset(
            data=data,
            i_filters=[
                FeatureFilter(keep_feature_names='target'),
            ],
        ),
        target_col_name='target',
    )

finetune_dm = PtlsDataModule(
    train_data=get_dataset(train),
    valid_data=get_dataset(valid),
    test_data=get_dataset(test),
    train_num_workers=4,
    train_batch_size=256,
    test_batch_size=128,
)

### Load pretrained TrxEncoder to new model

In [ ]:
import torch
import torchmetrics
from functools import partial
from ptls.nn import RnnSeqEncoder, Head
from ptls.frames.supervised import SequenceToTarget

In [ ]:
downstream_model = SequenceToTarget(
    seq_encoder=RnnSeqEncoder(
        trx_encoder=torch.nn.Sequential(
            torch.nn.Sequential(
                TrxEncoder(**trx_encoder_params),
                PBLinear(trx_encoder.output_size, 64),
                PBL2Norm(),
            ),
            PBLayerNorm(64),
        ),
        input_size=64,
        hidden_size=64,
        type='gru',
    ),
    head=Head(
        input_size=64,
        use_batch_norm=True,
        objective='classification',
        num_classes=4,
    ),
    loss=torch.nn.NLLLoss(),
    metric_list=torchmetrics.Accuracy(compute_on_step=False),
    pretrained_lr=0.005,
    optimizer_partial=partial(torch.optim.Adam, lr=0.015),
    lr_scheduler_partial=partial(torch.optim.lr_scheduler.StepLR, step_size=5, gamma=0.4),
)
downstream_model.seq_encoder.trx_encoder[0].load_state_dict(torch.load('mlm-emb.pt'))

In [ ]:
trainer_ft = pl.Trainer(
    max_epochs=3, # 32
    gpus=1 if torch.cuda.is_available() else 0,
    enable_progress_bar=False,
    callbacks=[
        pl.callbacks.LearningRateMonitor(logging_interval='step'),
    ]
)
print(f'logger.version = {trainer_ft.logger.version}')
trainer_ft.fit(downstream_model, finetune_dm)
print(trainer_ft.logged_metrics)
trainer_ft.test(dataloaders=finetune_dm.test_dataloader(), verbose=False)

### New model without pretrain

In [ ]:
downstream_model = SequenceToTarget(
    seq_encoder=RnnSeqEncoder(
        trx_encoder=TrxEncoder(**trx_encoder_params),
        hidden_size=64,
        type='gru',
    ),
    head=Head(
        input_size=64,
        use_batch_norm=False,
        objective='classification',
        num_classes=4,
    ),
    loss=torch.nn.NLLLoss(),
    metric_list=torchmetrics.Accuracy(compute_on_step=False),
    optimizer_partial=partial(torch.optim.Adam, lr=0.015),
    lr_scheduler_partial=partial(torch.optim.lr_scheduler.StepLR, step_size=5, gamma=0.5),
)

In [ ]:
trainer_ft = pl.Trainer(
    max_epochs=3,  # 32
    gpus=1 if torch.cuda.is_available() else 0,
    enable_progress_bar=False,
)
print(f'logger.version = {trainer_ft.logger.version}')
trainer_ft.fit(downstream_model, finetune_dm)
print(trainer_ft.logged_metrics)
trainer_ft.test(dataloaders=finetune_dm.test_dataloader())